# DATA 1030 Project Notebook - Part 1a, Create Target Columns
* The purpose of this notebook is to manipulate the raw data by converting time columns (initially measured as time taken, in seconds, to prove theorem) to columns of numerical labels, and export the new datasets for further use.
* We export three different files:
    1. Single-column target: labels 0 through 5
    2. 6-column target (strict equality): binary 0/1 labeling, 1 if heuristic gave min time, 0 otherwise
    3. 6-column target (loose equality): binary 0/1 labeling, 1 if heuristic gave min time + 0.01 sec tol, 0 otherwise
* Features and descriptions are also listed below for reference.

In [1]:
import pandas as pd
import numpy as np

### 1. Manually create feature names

* The dataset did not come with a header row, so I had to construct one.
* Referenced the original paper (Bridge et al) for the feature names and the five heuristics.
* The column labels are created below for the 14 static features S1 through S14 (columns 0-13) and the 39 dynamic features D1 through D39 (columns 14-52), and the time measurements from each heuristic for the row corresponding to a specific problem.

In [2]:
columns_type = ['S']*14 + ['D']*39
indices = [i for i in range (1,15)] + [i for i in range(1,40)]
H_Time = ['H1_Time', 'H2_Time', 'H3_Time', 'H4_Time','H5_Time']
type_join_number = [columns_type[i] + str(indices[i]) for i in range(0,53)] 
columns = type_join_number + H_Time

### 2. Read in the data and files for feature descriptions

In [3]:
df = pd.read_csv('../data/all-data-raw.csv', header=None, names=columns)

# Scraped PDF table and converted to CSV to retrieve the features from two tables in the paper
static_featnames = pd.read_csv('../data/static_features.csv') 
dynamic_featnames = pd.read_csv('../data/dynamic_features.csv')

full_features = pd.concat([static_featnames, dynamic_featnames], ignore_index=True)
full_features.drop('Feature number', axis= 1, inplace=True)
col_as_df = pd.DataFrame(columns, columns=['Feature Type and Number'])
full_features = col_as_df.merge(full_features, left_index=True, right_index=True)

### 3. Features and Descriptions
List of features is shown below.

In [4]:
full_features

,Feature Type and Number,Description
0,S1,Fraction of clauses that are unit clauses
1,S2,Fraction of clauses that are Horn clauses
2,S3,Fraction of clauses that are ground Clauses
3,S4,Fraction of clauses that are demodulators
4,S5,Fraction of clauses that are rewrite rules (or...
5,S6,Fraction of clauses that are purely positive
6,S7,Fraction of clauses that are purely negative
7,S8,Fraction of clauses that are mixed positive an...
8,S9,Maximum clause length
9,S10,Average clause length


###  4. Conversion of `H_Time` columns to target columns with binary classification labels
* The initial target coluns are not labels, but rather the time taken (in seconds) for each heuristic to prove the theorem/conjecture. 
* There was a time limit of 100 seconds.
* An entry of -100 denotes failure to obtain a proof within the time limit.
* `numpy.ma.MaskedArray` masks the -100 values to calculate the "true" min when one heuristic proved to be optimal.
* Target columns are converted here to a 0 through 5 labeled single column.
* This multiclass target column will be referred to as the "multi-class target".

In [5]:
df_heuristics_array = df[H_Time].values
H = np.zeros(shape=(df.shape[0], 6), dtype=np.int8)
y = np.zeros(shape=(df.shape[0],1), dtype=np.int8)
validate = ['']*df.shape[0] # To cross-reference with the author's data

In [6]:
print(df[H_Time].iloc[0:20]) # Some of our values

    H1_Time  H2_Time  H3_Time  H4_Time  H5_Time
0   -100.00  -100.00  -100.00  -100.00  -100.00
1      0.08     0.08     0.20     0.08     0.08
2   -100.00  -100.00  -100.00  -100.00  -100.00
3   -100.00  -100.00  -100.00  -100.00  -100.00
4   -100.00  -100.00  -100.00  -100.00  -100.00
5   -100.00  -100.00  -100.00  -100.00  -100.00
6   -100.00  -100.00  -100.00  -100.00  -100.00
7   -100.00  -100.00  -100.00  -100.00  -100.00
8   -100.00  -100.00  -100.00  -100.00  -100.00
9   -100.00  -100.00  -100.00  -100.00  -100.00
10  -100.00  -100.00  -100.00  -100.00  -100.00
11     0.06     0.06     0.08     0.06     0.07
12     0.06     0.06     0.08     0.06     0.06
13     0.12     0.10     0.07     0.11     0.12
14  -100.00  -100.00  -100.00  -100.00  -100.00
15  -100.00  -100.00  -100.00  -100.00  -100.00
16  -100.00  -100.00  -100.00  -100.00  -100.00
17  -100.00  -100.00  -100.00  -100.00  -100.00
18  -100.00  -100.00  -100.00  -100.00  -100.00
19  -100.00  -100.00  -100.00  -100.00  

In [7]:
best_nonH0 = np.ma.MaskedArray(df_heuristics_array, df_heuristics_array<0)

for index in range(df.shape[0]):  
    # If values are -100.0 in every HX_Time columns, then H0 (decline proof) is best heuristic (col 0 in H)
    if np.all(df_heuristics_array[index] == -100.0):
        H[index][0] = 1
        validate[index] = 'H0_Time' # validate array for confirming transformed results
    # Else: Mask values that are less than 0 and determine fastest heuristic with positive minimum time
    else:
        argmin = np.argmin(best_nonH0[index])
        # Insert value of 1 at index and location argmin + 1 (accounting for H0 in position 0)
        H[index][argmin+1] = 1
        y[index] = argmin + 1
        validate[index] = H_Time[argmin]

### 6. Validate conversions based on training data from author
* Before converting to the single-column target variable, it was important to check that my conversions are in line with the known values from the author-derived training data.
* The author had removed two of the features (Static feature 5 and dynamic feature 21) in his data so I removed these two from the `train_columns` index names as well, for column labeling to work properly.
* The two features were removed due to the fact that each one takes on the same value for all data points, as can be seen in the EDA of single columns above.

In [8]:
train_H_Time = ['H1_Time', 'H2_Time', 'H3_Time', 'H4_Time','H5_Time', 'H0_Time']
train_columns = [columns_type[i] + str(indices[i]) for i in range(0,53)] + train_H_Time 
train_columns.remove('S5')
train_columns.remove('D21')

df_train = pd.read_csv('../data/bridge-data/train.csv', header=None, names=train_columns)
df_H = df_train[train_H_Time]

for i in range(df_H.shape[0]):
    assert validate[i] == df_H.iloc[i].idxmax(axis='columns'), 'something is wrong at index {}'.format(i)
print('All results validated') # print if all assertions pass

All results validated


### 7. Construct single column target variable and Bridge target as DataFrames
* Labels $ Y \in \{ 0,1,2,3,4,5\} $ correspond to Heuristic Y being optimal for a specific row/data point.

In [9]:
target_multiple = pd.DataFrame(H, columns=['H0_Best','H1_Best', 'H2_Best', 'H3_Best', 'H4_Best', 'H5_Best'])
target_single = pd.DataFrame(y, columns=['Best Heuristic'])
df_target = target_multiple.merge(target_single, left_index=True, right_index=True)
df_target.head()

,H0_Best,H1_Best,H2_Best,H3_Best,H4_Best,H5_Best,Best Heuristic
0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,1
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0


### 8. Replace target columns with variable transformation in the raw data
* Merge on index

In [10]:
df_features = df.drop(H_Time, axis=1, inplace=False)
df_merged = df_features.merge(df_target, left_index=True, right_index=True)
df_merged.head()

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,D37,D38,D39,H0_Best,H1_Best,H2_Best,H3_Best,H4_Best,H5_Best,Best Heuristic
0,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.73872,0.073308,0.18797,1,0,0,0,0,0,0
1,0.83307,0.99682,0.83307,0.76948,0,0.77107,0.068363,0.16057,6,1.2734,...,0.74436,0.067669,0.18797,0,1,0,0,0,0,1
2,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.74248,0.069549,0.18797,1,0,0,0,0,0,0
3,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.73120,0.080827,0.18797,1,0,0,0,0,0,0
4,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.73308,0.078947,0.18797,1,0,0,0,0,0,0


### 9. Export data and merged features to CSV files

In [11]:
df_merged.to_csv('../data/multiclass_target_raw_data.csv', index=False)
full_features.to_csv('../data/features_plus_descriptions.csv', index=False)

### 10. Conversion of `H_Time` columns to multi-label binary encoded target columns

* Find all indices where there is a non-H0 optimal heuristic (i.e. all cases where an optimal heuristic was found).

In [12]:
df_heuristics_array = df[H_Time].values

nonH0_indices = []
for index in range(df_heuristics_array.shape[0]):
    if not np.all(df_heuristics_array[index] == -100.0):
        nonH0_indices.append(index)
print(len(nonH0_indices))

3564


### 11. Find all instances where there are multiple fastest heuristics
* Can have two, three, four, or five equal values out of five
* For reference, we can see how many of each group there are based on both strict and loose definitions of equality
* Strict equality: find all heuristics with exactly the same time as the min time
* Loose equality: find all heuristics with the min time + a tolerance level
    * Several small tolerances are tested below
    * Based on the results, I decided to go with the lowest and most conservative tolerance of 0.01 (seconds). I chose this because the number of results that end up in the "five-all-equal" bin grows very fast with increasing tolerance. If we increase the tolerance to even just 0.02, we end up with 500 more data points in the "five-all-equal" bin compared to baseline strict equality (we only have 84 data points then).

In [13]:
best_nonH0 = np.ma.MaskedArray(df_heuristics_array, df_heuristics_array < 0)

In [14]:
two_best = []
three_best = []
four_best = []
five_best = []
for index in nonH0_indices:
    row = best_nonH0[index]
    best_indices = np.argwhere(row==np.min(row))
    if best_indices.shape[0] == 2:
        two_best.append(index)
    elif best_indices.shape[0] == 3:
        three_best.append(index)
    elif best_indices.shape[0] == 4:
        four_best.append(index)
    elif best_indices.shape[0] == 5:
        five_best.append(index)
print("strict equality [row == np.min(row)] ")
print("     two equal:", len(two_best))
print("     three equal:", len(three_best))
print("     four equal:", len(four_best))
print("     five equal:", len(five_best))
print("     sum of all:", len(two_best+three_best+four_best+five_best))

strict equality [row == np.min(row)] 
     two equal: 485
     three equal: 191
     four equal: 143
     five equal: 84
     sum of all: 903


In [15]:
for atol in [0.01, 0.02, 0.04, 0.06, 0.08, 0.1]:
    two_best = []
    three_best = []
    four_best = []
    five_best = []
    for index in nonH0_indices:
        row = best_nonH0[index]
        best_indices = np.argwhere(np.isclose(row, np.min(row), atol=atol))
        if best_indices.shape[0] == 2:
            two_best.append(index)
        elif best_indices.shape[0] == 3:
            three_best.append(index)
        elif best_indices.shape[0] == 4:
            four_best.append(index)
        elif best_indices.shape[0] == 5:
            five_best.append(index)

    print("loose equality [with {} second grace-period] ".format(atol))
    print("     two equal:", len(two_best))
    print("     three equal:", len(three_best))
    print("     four equal:", len(four_best))
    print("     five equal:", len(five_best))
    print("     sum of all:", len(two_best+three_best+four_best+five_best))

loose equality [with 0.01 second grace-period] 
     two equal: 533
     three equal: 348
     four equal: 290
     five equal: 382
     sum of all: 1553
loose equality [with 0.02 second grace-period] 
     two equal: 464
     three equal: 357
     four equal: 320
     five equal: 572
     sum of all: 1713
loose equality [with 0.04 second grace-period] 
     two equal: 455
     three equal: 335
     four equal: 314
     five equal: 774
     sum of all: 1878
loose equality [with 0.06 second grace-period] 
     two equal: 453
     three equal: 339
     four equal: 318
     five equal: 879
     sum of all: 1989
loose equality [with 0.08 second grace-period] 
     two equal: 436
     three equal: 337
     four equal: 327
     five equal: 947
     sum of all: 2047
loose equality [with 0.1 second grace-period] 
     two equal: 438
     three equal: 347
     four equal: 323
     five equal: 1000
     sum of all: 2108


### 12. Create multi-label target columns
* For each row in the list of non-H0 indices, find all locations where the time value is equal to the minimum time, and add ones to the indicator matrix for all positive labels for that data point
* For each row in the list of non-H0 indices, find all locations where the time value is approximately the same as the minimum (tolerance = 0.5 sec) and add indicators for all positive labels for that data point

In [16]:
target = np.zeros((df.shape[0],6))
best_nonH0 = np.ma.MaskedArray(df_heuristics_array, df_heuristics_array < 0)

In [17]:
for index in range(df_heuristics_array.shape[0]):
    if np.all(df_heuristics_array[index] == -100.0):
        target[index][0] = 1
    else:
        row = best_nonH0[index]
        argmins = np.argwhere(row==np.min(row))
        for amin in argmins:
            target[index][amin+1] =  1
            

In [18]:
target_with_tol = np.zeros((df.shape[0],6))

In [19]:
for index in range(df_heuristics_array.shape[0]):
    if np.all(df_heuristics_array[index] == -100.0):
        target_with_tol[index][0] = 1
    else:
        row = best_nonH0[index]
        argmins = np.argwhere(np.isclose(row,np.min(row), atol=0.01))
        for amin in argmins:
            target_with_tol[index][amin+1] =  1
            

In [20]:
target = pd.DataFrame(target.astype(int), columns=['H0_Best','H1_Best', 'H2_Best', 'H3_Best', 'H4_Best', 'H5_Best'])
target_with_tol = pd.DataFrame(target_with_tol.astype(int), columns=['H0_Best','H1_Best', 'H2_Best', 'H3_Best', 'H4_Best', 'H5_Best'])
df_features = df.drop(H_Time, axis=1, inplace=False)
df_merged_strict = df_features.merge(target, left_index=True, right_index=True)
df_merged_with_tol = df_features.merge(target_with_tol, left_index=True, right_index=True)

In [21]:
df_merged_strict.head()

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,D36,D37,D38,D39,H0_Best,H1_Best,H2_Best,H3_Best,H4_Best,H5_Best
0,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.00188,0.73872,0.073308,0.18797,1,0,0,0,0,0
1,0.83307,0.99682,0.83307,0.76948,0,0.77107,0.068363,0.16057,6,1.2734,...,0.00188,0.74436,0.067669,0.18797,0,1,1,0,1,1
2,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.00188,0.74248,0.069549,0.18797,1,0,0,0,0,0
3,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.00188,0.73120,0.080827,0.18797,1,0,0,0,0,0
4,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.00188,0.73308,0.078947,0.18797,1,0,0,0,0,0


In [22]:
df_merged_with_tol.head()

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,D36,D37,D38,D39,H0_Best,H1_Best,H2_Best,H3_Best,H4_Best,H5_Best
0,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.00188,0.73872,0.073308,0.18797,1,0,0,0,0,0
1,0.83307,0.99682,0.83307,0.76948,0,0.77107,0.068363,0.16057,6,1.2734,...,0.00188,0.74436,0.067669,0.18797,0,1,1,0,1,1
2,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.00188,0.74248,0.069549,0.18797,1,0,0,0,0,0
3,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.00188,0.73120,0.080827,0.18797,1,0,0,0,0,0
4,0.83307,0.99682,0.83307,0.76789,0,0.76948,0.069952,0.16057,6,1.2734,...,0.00188,0.73308,0.078947,0.18797,1,0,0,0,0,0


### 13. Export target columns and merged data (strict and 0.01 tolerance targets) to CSV files

In [23]:
df_merged_strict.to_csv('../data/multilabel_raw_data_strict.csv', index=False)
df_merged_with_tol.to_csv('../data/multilabel_raw_data_tol.csv', index=False)